Time Calibration of the Mu2e Calorimeter

by Giacinto boccia

version 0.1 | 2024-09-05

In [77]:
import numpy as np
import awkward as ak
import uproot
import quantities as pq
from concurrent.futures import ProcessPoolExecutor

In [78]:
#Cut parameters
HITNUM_CUT = 1
Q_MIN_CUT = 4000
Q_MAX_CUT = 8000
COS_THETA_CUT = 0.9
CHI_ON_NDF_CUT = 2000

In [79]:
#Input
hits_path = input("Hits file to process:")
cal_path = input("Starting caibration file:") or False
n_runs = input("Iterations to perform:")
#Name of the tree inside the file
hits_path += ":sidet"

In [80]:
#Opening the calibration start-point file
if cal_path:
    print("Starting calibration file not yet supported")
    cal_corection = pq.Quantity(np.zeros((36, 28, 2)), 'ns')
else:
    #The time-correction array is [rows, columns, SiPM] shaped
    cal_corection = pq.Quantity(np.zeros((36, 28, 2)), 'ns')

In [99]:
#Loading the tree, start by defining custom event class that can compute residurals
class Cosmic(ak.Record):     
    def t_residuals(self) -> dict | None:
        #Filters on the hit charge 
        q_min_fitler = self.Qval > Q_MIN_CUT
        q_max_fitler = self.Qval < Q_MAX_CUT
        q_filter = q_max_fitler & q_min_fitler
        if self.slope:
            #Select events that have a not "None" slope. For each hit return the time residual
            cos_theta = 1 / np.sqrt(1 + self.slope ** 2)
            t_arr = pq.Quantity(np.array(self.Tval[q_filter] + self.templTime[q_filter]), 'ns')
            t_res = dict()
            for i, [row, col, sipm] in enumerate(zip(self.iRow[q_filter],
                                                     self.iCol[q_filter],
                                                     self.SiPM[q_filter])):
                #Apply current time corrections
                t_arr[i] -= cal_corection[row, col, sipm]
            print(t_arr)
            t_0_ev = np.average(t_arr, )#weights= self.Qval[q_filter]
            for time, y, row, col, sipm in zip(t_arr, 
                                               self.Yval[q_filter],
                                               self.iRow[q_filter],
                                               self.iCol[q_filter],
                                               self.SiPM[q_filter]):
                #Each residual is stored in the dictionary with (row, col, sipm) as key
                y = y * pq.cm
                print(t_0_ev)
                print(time)
                t_res[(row, col, sipm)] = time + y / (pq.c * cos_theta) - t_0_ev
            return t_res
        else:
            return None                
ak.behavior["cosmic"] = Cosmic
        
#Loading tree in an array structure, we only need some of the branches
branches = ("nrun", "nsubrun", "evnum", "nHits", "iRow", "iCol", "SiPM", "Xval", "Yval", "Qval", "Tval", "templTime")
with uproot.open(hits_path) as file:
    tree = file.arrays(filter_name = branches, entry_stop= 1000)
    
#Now change the array so that it uses the custom class defned above
tree = ak.Array(tree, with_name= "cosmic")

In [100]:
#To get the parameters, each event is fitted
def linear_fit(event) -> dict[str, np.double | int]:
    q_min_fitler = event.Qval > Q_MIN_CUT
    q_max_fitler = event.Qval < Q_MAX_CUT
    x_arr = event.Xval[q_min_fitler & q_max_fitler]
    y_arr = event.Yval[q_min_fitler & q_max_fitler]
    if len(x_arr) > 1:
        #Events that are not empty
        if np.max(x_arr) - np.min(x_arr) > 34.4:
            #Events that are not vertical
            [slope, intercept], residuals, _, _, _ = np.polyfit(x_arr, y_arr, deg= 1, full= True)
            chi_sq : np.double = np.sum(residuals)
            ndf = event.nHits - 2
            return {'vertical' : False, 'slope' : slope, 'intercept' : intercept, 'chi_sq' : chi_sq, 'ndf' : ndf}
        else:
            #Vertical events get flagged
            return {'vertical': True, 'slope' : None, 'intercept' : None, 'chi_sq' : None, 'ndf' : None}
    else:
        return None
        
with ProcessPoolExecutor() as executor:
    fit_results = list(executor.map(linear_fit, tree))
fit_results = ak.Array(fit_results)

In [101]:
#Add the fit results to the tree
if hasattr(fit_results, 'vertical'):
    tree = ak.with_field(tree, fit_results.vertical, "vertical")
    tree = ak.with_field(tree, fit_results.slope, "slope")
    tree = ak.with_field(tree, fit_results.intercept, "intercept")
    tree = ak.with_field(tree, fit_results.chi_sq, "chi_sq")
    tree = ak.with_field(tree, fit_results.ndf, "ndf")

In [102]:
tree.Qval

<Array [[1.06e+04, 1.02e+04, ..., 9.63e+03], ...] type='1000 * var * float64'>

In [103]:
#Apply the filters and compute the residuals
slope_cut = np.sqrt(1 / COS_THETA_CUT - 1)
filters = {'n_min' : tree.nHits > HITNUM_CUT,
           'slope_min' : abs(tree.slope) > slope_cut,
           'chi_sq' : tree.chi_sq / tree.ndf < CHI_ON_NDF_CUT}
filtered_events = tree[filters['n_min'] & filters['slope_min'] & filters['chi_sq']]
filtered_events = ak.drop_none(filtered_events)
def get_t_residuals(event) -> dict | None:
    return event.t_residuals()

#with ProcessPoolExecutor() as executor:
#    residuals = list(executor.map(get_t_residuals, filtered_events))
for i, event in enumerate(filtered_events):
    residuals = event.t_residuals()
    print("Event ", i, residuals)

[79307.83291626 79303.12965012 79301.7393837 ] ns
79304.23398335774 ns
79307.83291625977 ns
79304.23398335774 ns
79303.12965011597 ns
79304.23398335774 ns
79301.73938369751 ns
Event  0 {(np.int32(6), np.int32(25), np.int32(1)): array(-14.48069737) * ns, (np.int32(6), np.int32(25), np.int32(0)): array(-19.18396352) * ns, (np.int32(9), np.int32(9), np.int32(0)): array(-16.0543227) * ns}
[90796.17483139 90795.03450394 90794.35357666 90795.75474167] ns
90795.329413414 ns
90796.17483139038 ns
90795.329413414 ns
90795.03450393677 ns
90795.329413414 ns
90794.35357666016 ns
90795.329413414 ns
90795.7547416687 ns
Event  1 {(np.int32(22), np.int32(9), np.int32(0)): array(7.31756533) * ns, (np.int32(24), np.int32(12), np.int32(0)): array(9.41331192) * ns, (np.int32(22), np.int32(9), np.int32(1)): array(5.4963106) * ns, (np.int32(24), np.int32(12), np.int32(1)): array(10.13354965) * ns}
[75694.4311676  75696.13565063 75692.9099617  75692.91303635] ns
75694.09745407104 ns
75694.43116760254 ns
75694

/tmp/ipykernel_3483/1448941455.py:11: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  t_arr = pq.Quantity(np.array(self.Tval[q_filter] + self.templTime[q_filter]), 'ns')


7022.564721425374 ns
7019.767875671387 ns
7022.564721425374 ns
7026.517379760742 ns
7022.564721425374 ns
7025.866386413574 ns
7022.564721425374 ns
7021.290451049805 ns
7022.564721425374 ns
7021.16813659668 ns
Event  10 {(np.int32(16), np.int32(10), np.int32(0)): array(-7.78275368) * ns, (np.int32(16), np.int32(10), np.int32(1)): array(-8.79297707) * ns, (np.int32(6), np.int32(25), np.int32(1)): array(-32.02413087) * ns, (np.int32(4), np.int32(22), np.int32(1)): array(-38.6712562) * ns, (np.int32(4), np.int32(22), np.int32(0)): array(-43.24719156) * ns, (np.int32(5), np.int32(23), np.int32(0)): array(-40.37143869) * ns}
[63995.67566681 63995.9682312  63998.65994263 63995.1202507
 63995.44023895 63994.87496567] ns
63995.956549326576 ns
63995.67566680908 ns
63995.956549326576 ns
63995.96823120117 ns
63995.956549326576 ns
63998.65994262695 ns
63995.956549326576 ns
63995.120250701904 ns
63995.956549326576 ns
63995.44023895264 ns
63995.956549326576 ns
63994.874965667725 ns
Event  11 {(np.int

In [ ]:
residuals